Consider
$$
-\nabla \cdot \left(\kappa(x,y) \nabla u (x,y) \right) = f
$$
defined over a unit square, $\Omega$. This equation might describe heat conduction. Where $\kappa(x,y) \geq 0$ is a variable heat conductivity. 

Let's partition $\Omega$ into two subdomains, $\Omega_{0} = [0,1] \times [0,1/2]$ and $\Omega_{1} = [0,1] \times (1/2, 1]$.

For our purposes, we consider $\kappa(\Omega_{0}) = \kappa_{0}$, and $\kappa(\Omega_{1}) = \kappa_{1}$. These are constants.

The infinite dimensional weak form is
$$
\int_{\Omega} \kappa \left(\nabla u \cdot \nabla v\right)\,dx = \int_{\Omega} fv\,dx
$$

In [4]:
from fenics import *
from mshr import *

The main challenge comes from defining $\kappa$. We show several ways to do this.

1. Subclassing ``Expression`` 

```python
class K(Expression):

    def set_k_values(self, k_0, k_1):
        self.k_0 = k_0
        self.k_1 = k_1
        
    def eval(self, value, x):
        tol = 1E-14
        # if y <= 0.5, so we are in Omega_0
        if x[1] <= 0.5 + tol:
            value[0] = self.k_0
        else:
            value[0] = self.k_1

kappa = K()
kappa.set_k_values(1, 0.01)
```
But **FEniCS will call ``eval`` for every node**. (Maybe we are overloading eval here?)

2. Using a C++ string. This works for simple $\kappa$
```
tol = 1E-14
k_0 = 1.0
k_1 = 0.01
kappa = Expression("x[1] <= 0.5 + tol ? k_0 : k_1", 
                    degree = 0, 
                    tol = tol, 
                    k_0 = k_0, 
                    k_1 = k_1)
```

More general techniques exist. We will get to them later.

The boundary function we have been defining is a shortcut for a more general concept called `Subdomain`.

A `Subdomain` defines a region in space in terms of a special method, called `inside`. `inside` returns `True` for all points that belong to the subdomain, and false otherwise.

For our case, we define two subdomains.

In [5]:
class Omega_0(SubDomain):
    def inide(self, x, on_boundary):
        return x[1] <= 0.5 + tol

class Omega_1(SubDomain):
    def inide(self, x, on_boundary):
        return x[1] >= 0.5 - tol